### train file and eval file


In [ ]:
import csv
import json
from collections import defaultdict

with open(r"E:\badouai\ai\week7\week7 文本分类问题\文本分类练习.csv", "r", encoding="utf-8") as f:
    reader = csv.reader(f)
    data = list(reader)[1:]
    data1 = defaultdict(list)
    data_train = defaultdict(list)
    data_eval = defaultdict(list)
    for single_list in data:
        data1[int(single_list[0])] += [single_list[1]]
    for label, value in data1.items():
        if label == 0:
            data_eval["差评"] = value[:100]
            data_train["差评"] = value[100:]
        if label == 1:
            data_eval["好评"] = value[:100]
            data_train["好评"] = value[100:]
with open(r"E:\badouai\ai\week7\homework\train.json", "w", encoding="utf-8") as f:
    first = True  # 用于跟踪是否是第一个字典项
    for key, value in data_train.items():
        for line in value:
            # add_dict = {"tag": "差评" if key == 0 else "好评", "title": line}
            if key == "差评":
                add_dict = {"tag":"差评", "title": line}
            elif key == "好评":
                add_dict = {"tag":"好评", "title": line}
            json_str = json.dumps(add_dict, ensure_ascii=False)  # 转换为 JSON 字符串
            # 如果这不是第一个字典项，先写入一个换行符
            if not first:
                f.write('\n')
            else:
                first = False
            f.write(json_str)  # 写入 JSON 字符串

with open(r"E:\badouai\ai\week7\homework\eval.json", "w", encoding="utf-8") as f:
    first = True  # 用于跟踪是否是第一个字典项
    for key, value in data_eval.items():
        for line in value:
            # add_dict = {"tag": "差评" if key == 0 else "好评", "title": line}
            if key == "差评":
                add_dict = {"tag":"差评", "title": line}
            elif key == "好评":
                add_dict = {"tag":"好评", "title": line}
            json_str = json.dumps(add_dict, ensure_ascii=False)  # 转换为 JSON 字符串
            # 如果这不是第一个字典项，先写入一个换行符
            if not first:
                f.write('\n')
            else:
                first = False
            f.write(json_str)  # 写入 JSON 字符串


### Config

In [ ]:
# -*- coding: utf-8 -*-

"""
配置参数信息
"""

Config = {
    "model_path": r"E:\badouai\ai\week7\homework",
    "train_data_path": r"E:\badouai\ai\week7\homework\train.json",
    "valid_data_path": r"E:\badouai\ai\week7\homework\eval.json",
    "vocab_path":r"E:\badouai\ai\week7\week7 文本分类问题\nn_pipline\chars.txt",
    "model_type":"bert",
    "max_length": 30,
    "hidden_size": 256,
    "kernel_size": 3,
    "num_layers": 2,
    "epoch": 15,
    "batch_size": 128,
    "pooling_style":"max",
    "optimizer": "adam",
    "learning_rate": 1e-3,
    "pretrain_model_path":r"E:\badouai\ai\week6\bert-base-chinese",
    "seed": 987
}



### loader

In [ ]:
# -*- coding: utf-8 -*-

import json
import re
import os
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer
"""
数据加载
"""


class DataGenerator:
    def __init__(self, data_path, config):
        self.config = config
        self.path = data_path
        self.index_to_label = {0: '差评', 1: '好评'}
        self.label_to_index = dict((y, x) for x, y in self.index_to_label.items())
        self.config["class_num"] = len(self.index_to_label)
        if self.config["model_type"] == "bert":
            self.tokenizer = BertTokenizer.from_pretrained(config["pretrain_model_path"])
        self.vocab = load_vocab(config["vocab_path"])
        self.config["vocab_size"] = len(self.vocab)
        self.load()


    def load(self):
        self.data = []
        with open(self.path, encoding="utf8") as f:
            for line in f:
                line = json.loads(line)
                tag = line["tag"]
                label = self.label_to_index[tag]
                title = line["title"]
                if self.config["model_type"] == "bert":
                    input_id = self.tokenizer.encode(title, max_length=self.config["max_length"], pad_to_max_length=True)
                else:
                    input_id = self.encode_sentence(title)
                input_id = torch.LongTensor(input_id)
                label_index = torch.LongTensor([label])
                self.data.append([input_id, label_index])
        return

    def encode_sentence(self, text):
        input_id = []
        for char in text:
            input_id.append(self.vocab.get(char, self.vocab["[UNK]"]))
        input_id = self.padding(input_id)
        return input_id

    #补齐或截断输入的序列，使其可以在一个batch内运算
    def padding(self, input_id):
        input_id = input_id[:self.config["max_length"]]
        input_id += [0] * (self.config["max_length"] - len(input_id))
        return input_id

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data[index]

def load_vocab(vocab_path):
    token_dict = {}
    with open(vocab_path, encoding="utf8") as f:
        for index, line in enumerate(f):
            token = line.strip()
            token_dict[token] = index + 1  #0留给padding位置，所以从1开始
    return token_dict


#用torch自带的DataLoader类封装数据
def load_data(data_path, config, shuffle=True):
    dg = DataGenerator(data_path, config)
    dl = DataLoader(dg, batch_size=config["batch_size"], shuffle=shuffle)
    return dl

if __name__ == "__main__":
    from config import Config
    dg = DataGenerator("valid_tag_news.json", Config)
    print(dg[1])


### main

In [ ]:
# -*- coding: utf-8 -*-

import torch
import os
import random
import os
import csv
import time
import numpy as np
import logging
from config import Config
from model import TorchModel, choose_optimizer
from evaluate import Evaluator
from loader import load_data
#[DEBUG, INFO, WARNING, ERROR, CRITICAL]
logging.basicConfig(level=logging.INFO, format = '%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

"""
模型训练主程序
"""


seed = Config["seed"] 
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

def main(config):
    #创建保存模型的目录
    if not os.path.isdir(config["model_path"]):
        os.mkdir(config["model_path"])
    #加载训练数据
    train_data = load_data(config["train_data_path"], config)
    #加载模型
    model = TorchModel(config)
    # 标识是否使用gpu
    cuda_flag = torch.cuda.is_available()
    if cuda_flag:
        logger.info("gpu可以使用，迁移模型至gpu")
        model = model.cuda()
    #加载优化器
    optimizer = choose_optimizer(config, model)
    #加载效果测试类
    evaluator = Evaluator(config, model, logger)
    #训练
    for epoch in range(config["epoch"]):
        epoch += 1
        model.train()
        logger.info("epoch %d begin" % epoch)
        train_loss = []
        for index, batch_data in enumerate(train_data):
            if cuda_flag:
                batch_data = [d.cuda() for d in batch_data]

            optimizer.zero_grad()
            input_ids, labels = batch_data   #输入变化时这里需要修改，比如多输入，多输出的情况
            loss = model(input_ids, labels)
            loss.backward()
            optimizer.step()

            train_loss.append(loss.item())
            if index % int(len(train_data) / 2) == 0:
                logger.info("batch loss %f" % loss)
        logger.info("epoch average loss: %f" % np.mean(train_loss))
        acc = evaluator.eval(epoch)
    # model_path = os.path.join(config["model_path"], "epoch_%d.pth" % epoch)
    # torch.save(model.state_dict(), model_path)  #保存模型权重
    return acc

if __name__ == "__main__":
    # main(Config)

    # for model in ["cnn"]:
    #     Config["model_type"] = model
    #     print("最后一轮准确率：", main(Config), "当前配置：", Config["model_type"])

    #对比所有模型
    #中间日志可以关掉，避免输出过多信息
    # 超参数的网格搜索
    with open(r"E:\badouai\ai\week7\homework\result.csv", "w", encoding="utf-8") as f:
        writer = csv.writer(f)
        header = ["model_type", "learning_rate", "hidden_size", "batch_size", "pooling_style", "acc", "time"]
        writer.writerow(header)
        for model in ["cnn", "rnn", "bert", "lstm", "gru", "gated_cnn", "rcnn"]:
            Config["model_type"] = model
            start_time = time.time()
            print(f"模型：{model}，开始时间：{start_time}")
            for lr in [1e-3, 1e-4]:
                Config["learning_rate"] = lr
                if Config["model_type"] == "bert":
                    hidden_size_list = [768]
                else:
                    hidden_size_list = [128, 256]
                for hidden_size in hidden_size_list:
                    Config["hidden_size"] = hidden_size
                    for batch_size in [64, 128]:
                        Config["batch_size"] = batch_size
                        for pooling_style in ["avg", "max"]:
                            Config["pooling_style"] = pooling_style
                            print("最后一轮准确率：", main(Config), "当前配置：", Config)
                            writer.writerow([model, lr, hidden_size, batch_size, pooling_style, main(Config), time.time() - start_time])


### result

model_type	learning_rate	hidden_size	batch_size	pooling_style	acc	time
                        
cnn	0.001	128	64	avg	0.855	22.32502961
                        
cnn	0.001	128	64	max	0.86	35.5318656
                        
cnn	0.001	128	128	avg	0.85	44.88531137
                        
cnn	0.001	128	128	max	0.865	54.61551237
                        
cnn	0.001	256	64	avg	0.845	75.09645581
                        
cnn	0.001	256	64	max	0.845	96.32786369
                        
cnn	0.001	256	128	avg	0.855	111.0196004
                        
cnn	0.001	256	128	max	0.845	126.5093691
                        
cnn	0.0001	128	64	avg	0.88	139.3278177
                        
cnn	0.0001	128	64	max	0.87	152.4910762
                        
cnn	0.0001	128	128	avg	0.835	161.7517056
                        
cnn	0.0001	128	128	max	0.875	171.3785837
                        
cnn	0.0001	256	64	avg	0.865	191.924191
                        
cnn	0.0001	256	64	max	0.895	213.1848226
                        
cnn	0.0001	256	128	avg	0.85	228.111635
                        
cnn	0.0001	256	128	max	0.875	243.5794377
                        
rnn	0.001	128	64	avg	0.86	17.73826623
                        
rnn	0.001	128	64	max	0.87	34.32513356
                        
rnn	0.001	128	128	avg	0.865	46.4845221
                        
rnn	0.001	128	128	max	0.875	59.03364468
                        
rnn	0.001	256	64	avg	0.86	91.94913483
                        
rnn	0.001	256	64	max	0.855	125.7109878
                        
rnn	0.001	256	128	avg	0.85	154.4917631
                        
rnn	0.001	256	128	max	0.865	184.1074591
                        
rnn	0.0001	128	64	avg	0.87	200.4476593
                        
rnn	0.0001	128	64	max	0.875	217.1574116
                        
rnn	0.0001	128	128	avg	0.86	229.2438855
                        
rnn	0.0001	128	128	max	0.885	241.7149432
                        
rnn	0.0001	256	64	avg	0.88	274.6851103
                        
rnn	0.0001	256	64	max	0.865	308.6718249
                        
rnn	0.0001	256	128	avg	0.87	337.4615014
                        
rnn	0.0001	256	128	max	0.87	367.1906652
                        
bert	0.001	768	64	avg	0.86	335.9281619
                        
bert	0.001	768	64	max	0.865	670.910861
                        
bert	0.001	768	128	avg	0.86	966.188699
                        
bert	0.001	768	128	max	0.865	1260.317307
                        
bert	0.0001	768	64	avg	0.87	1595.422981
                        
bert	0.0001	768	64	max	0.89	1929.996548
                        
bert	0.0001	768	128	avg	0.875	2224.901199
                        
bert	0.0001	768	128	max	0.89	2519.612361
                        
lstm	0.001	128	64	avg	0.89	31.31020689
                        
lstm	0.001	128	64	max	0.875	62.71541619
                        
lstm	0.001	128	128	avg	0.865	90.95092082
                        
lstm	0.001	128	128	max	0.885	119.7199686
                        
lstm	0.001	256	64	avg	0.875	182.8570802
                        
lstm	0.001	256	64	max	0.875	246.7106535
                        
lstm	0.001	256	128	avg	0.88	302.663357
                        
lstm	0.001	256	128	max	0.83	359.1400239
                        
lstm	0.0001	128	64	avg	0.885	389.5645416
                        
lstm	0.0001	128	64	max	0.87	420.3412535
                        
lstm	0.0001	128	128	avg	0.88	448.2398386
                        
lstm	0.0001	128	128	max	0.89	476.4472737
                        
lstm	0.0001	256	64	avg	0.85	539.4177291
                        
lstm	0.0001	256	64	max	0.885	603.303839
                        
lstm	0.0001	256	128	avg	0.87	659.3078237
                        
lstm	0.0001	256	128	max	0.9	716.0215955
                        
gru	0.001	128	64	avg	0.87	23.07916331
                        
gru	0.001	128	64	max	0.875	46.25065613
                        
gru	0.001	128	128	avg	0.86	66.63330102
                        
gru	0.001	128	128	max	0.9	87.60907507
                        
gru	0.001	256	64	avg	0.9	142.9116254
                        
gru	0.001	256	64	max	0.875	199.2451489
                        
gru	0.001	256	128	avg	0.87	246.8751929
                        
gru	0.001	256	128	max	0.875	295.3396363
                        
gru	0.0001	128	64	avg	0.885	318.43646
                        
gru	0.0001	128	64	max	0.86	341.7463377
                        
gru	0.0001	128	128	avg	0.86	361.912529
                        
gru	0.0001	128	128	max	0.88	382.6267259
                        
gru	0.0001	256	64	avg	0.9	438.1744163
                        
gru	0.0001	256	64	max	0.88	494.541379
                        
gru	0.0001	256	128	avg	0.865	542.051676
                        
gru	0.0001	256	128	max	0.87	590.4650757
                        
gated_cnn	0.001	128	64	avg	0.855	15.45683718
                        
gated_cnn	0.001	128	64	max	0.865	31.14057398
                        
gated_cnn	0.001	128	128	avg	0.875	43.1635766
                        
gated_cnn	0.001	128	128	max	0.87	55.65457535
                        
gated_cnn	0.001	256	64	avg	0.865	87.34765458
                        
gated_cnn	0.001	256	64	max	0.88	119.5271471
                        
gated_cnn	0.001	256	128	avg	0.87	142.6221111
                        
gated_cnn	0.001	256	128	max	0.855	166.1181951
                        
gated_cnn	0.0001	128	64	avg	0.885	181.501816
                        
gated_cnn	0.0001	128	64	max	0.875	196.9651101
                        
gated_cnn	0.0001	128	128	avg	0.87	209.1940687
                        
gated_cnn	0.0001	128	128	max	0.89	221.6699221
                        
gated_cnn	0.0001	256	64	avg	0.85	253.3007152
                        
gated_cnn	0.0001	256	64	max	0.9	285.5411942
                        
gated_cnn	0.0001	256	128	avg	0.885	308.5973182
                        
gated_cnn	0.0001	256	128	max	0.875	332.1804705
                        
rcnn	0.001	128	64	avg	0.84	19.4196651
                        
rcnn	0.001	128	64	max	0.84	40.13133216
                        
rcnn	0.001	128	128	avg	0.865	54.81967974
                        
rcnn	0.001	128	128	max	0.88	69.92298007
                        
rcnn	0.001	256	64	avg	0.86	111.6694474
                        
rcnn	0.001	256	64	max	0.85	153.8939385
                        
rcnn	0.001	256	128	avg	0.855	185.7154634
                        
rcnn	0.001	256	128	max	0.87	217.971786
                        
rcnn	0.0001	128	64	avg	0.87	237.4363618
                        
rcnn	0.0001	128	64	max	0.89	258.2464113
                        
rcnn	0.0001	128	128	avg	0.875	273.0485137
                        
rcnn	0.0001	128	128	max	0.875	288.1557209
                        
rcnn	0.0001	256	64	avg	0.86	330.1321905
                        
rcnn	0.0001	256	64	max	0.865	372.6673522
                        
rcnn	0.0001	256	128	avg	0.875	404.3369019
                        
rcnn	0.0001	256	128	max	0.89	436.5389478
